# 1. Hệ mật RSA 512 bits

In [8]:
! pip install pycryptodome
! pip install Crypto --user

In [10]:
from Crypto.Util.number import bytes_to_long, long_to_bytes
from Crypto.Random import get_random_bytes 
import Crypto
import libnum

msg="hoailam"

# Khoa cong khai (n, e) khoa rieng tu d
p = Crypto.Util.number.getPrime(512, randfunc=get_random_bytes)
q = Crypto.Util.number.getPrime(512, randfunc=get_random_bytes)

n = p*q
phi=(p-1)*(q-1)

e=65537
d=libnum.invmod(e,phi)

m=  bytes_to_long(msg.encode('utf-8'))

cipher=pow(m,e, n)
decipher=pow(cipher,d ,n)

print ("Message=%s\np=%s\nq=%s\n\nd=%d\ne=%d\nN=%s\n\nMa_hoa=%s\n\nGiai_ma=%s" % (msg,p,q,d,e,n,cipher,(long_to_bytes(decipher))))

Message=hoailam
p=12361100961649939059433382920395959210739948344599594825808827407913931324356940173861694642574350265545441684242675148882618440123143870718768234974396611
q=12703064859867041018095629711087561562235860205478517149089620871998374149226896211434793950803355787498478343647468084664476125006756517245729983483422511

d=38634814829640736821081507214484800404593483720473336923579817446981035028587396559891143626771595065637983717022628865470310819285930081016522794292768027290612970308463470165409736518289128136796636918792318389321377241680715918114108289414605676083400853484306445886437876608957485666666403042134629060973
e=65537
N=157023867255204029087951549662988549712610427447358826788257395102188904971691919897400674720479381445997924890698420338376915358979348819818905697337372936973927200438268167093692191309822453019296376046911489334476503185981695545104249790340687157146539353054973102177929926587355625288686634648443399510221

Ma_hoa=1018001998642489814408423005

# 2. Hệ mật Elgamal 256 bits

In [11]:
from random import randint

# Khoa cong khai (p, alpha, beta), khoa rieng tu a
p = Crypto.Util.number.getPrime(256, randfunc=get_random_bytes)
alpha = 2
k = randint(0, p-1)
a = 765
beta = pow(alpha, a, p)

msg = "hoailam"
m=  bytes_to_long(msg.encode('utf-8'))

y1 = pow(alpha, k, p)
y2 = (pow(beta, k, p) * (m%p)) % p

decipher = ((y2%p) * libnum.invmod(pow(y1,a), p))%p

print("Message=%s\np=%s\nAlpha=%s\na=%s\nBeta=%s\nk=%s\n\nMa_hoa:\n\ty1=%s\n\ty2=%s\n\nGiai_ma=%s" % (msg,p,alpha,a,beta,k,y1,y2,(long_to_bytes(decipher))))

Message=hoailam
p=95757006111623635108841940793057106450280418779972246374742501921893115493211
Alpha=2
a=765
Beta=63999578118578642493535910320147124563537627748227240577849982963620043493728
k=18212636745836060366641735434260137212286604724402705047555116720990749677059

Ma_hoa:
	y1=76236373163793537168763218880738010203114063121642655961617051338393042731939
	y2=41844859692105554402084906452261103532824553359767747388627925007330072821058

Giai_ma=b'hoailam'


# 3. Hệ mật trên đường cong Elliptic (EC-Elgamal với p = 50 bits)

Chọn đường cong elliptic sau: y^2 = x^3 + x + 1

In [12]:
p = Crypto.Util.number.getPrime(50, randfunc=get_random_bytes)

## y^2 = x^3 + x + 1
a, b = 1, 1

curve = libnum.ecc.Curve(a, b, p)
## first 50 points
print(curve.find_points_in_range(1,50))

[(3, 296740683462827), (3, 597984024761876), (4, 641025303813169), (4, 253699404411534), (5, 137914070927020), (5, 756810637297683), (6, 612195532744579), (6, 282529175480124), (7, 749259186000990), (7, 145465522223713), (12, 122510770477028), (12, 772213937747675), (13, 457305230175154), (13, 437419478049549), (16, 299431616562625), (16, 595293091662078), (19, 139277471409405), (19, 755447236815298), (20, 473931581056851), (20, 420793127167852), (23, 872388318323167), (23, 22336389901536), (26, 113456120950406), (26, 781268587274297), (27, 360566927810904), (27, 534157780413799), (29, 620048784288329), (29, 274675923936374), (33, 140813792971409), (33, 753910915253294), (34, 593464600574953), (34, 301260107649750), (35, 866668448175257), (35, 28056260049446), (36, 731869605967120), (36, 162855102257583), (37, 261256415193149), (37, 633468293031554), (38, 868262722555780), (38, 26461985668923), (39, 150343838546153), (39, 744380869678550), (40, 673974120283126), (40, 220750587941577), 

In [26]:
P = curve.check_x(76)
print(list(P))

[(76, 225579937093506), (76, 669144771131197)]


In [50]:
P = (76, 225579937093506)
s = 947
## B = sP
B = curve.power(P, s)

M = (3, 296740683462827)
k = 2908

## cipher
M1 = curve.power(P, k)
M2 = curve.add(M, curve.power(B, k))

## decipher
M1_1 = curve.power(M1, s)
M1_2 = (M1_1[0], -M1_1[1])
decipher = curve.add(M2, M1_2)

print("p=", p)
print("Message=", M)
print("Khoa_cong_khai:\n\tE:y^2=x^3+%sx+%s" % (a,b))
print("\tP=", P)
print("\tB=", B)
print("Khoa_rieng_tu: s=", s)
print("Ma_hoa:")
print("\tk=", k)
print("\tM1=", M1)
print("\tM2=", M2)
print("Giai_ma=", decipher)


p= 894724708224703
Message= (3, 296740683462827)
Khoa_cong_khai:
	E:y^2=x^3+1x+1
	P= (76, 225579937093506)
	B= (709105722669586, 555767849562817)
Khoa_rieng_tu: s= 947
Ma_hoa:
	k= 2908
	M1= (637004803715825, 496064363601268)
	M2= (398382499393501, 422748624162773)
Giai_ma= (3, 296740683462827)


# Nguyễn Hoài Lâm - 18020748 - INT3213 1